# Topic Modeling

## Introduction
Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do 
**Latent Dirichlet Allocation(LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide(1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model

## Topic Modeling #1 (All Text) 

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle
data = pd.read_pickle('dtm_stop.pkl')
data

,abilities,ability,able,abnormal,aboriginal,abounds,abrupt,abruzzi,absence,absorption,...,yoked,yoking,york,young,youth,youthful,zeitgeist,zoo,zoology,émigrés
Ernest Hemingway,0,1,4,0,0,0,0,2,0,0,...,0,0,3,8,0,0,1,0,0,0
Joseph Conrad,0,0,0,0,1,1,0,0,1,1,...,1,1,0,0,1,0,0,0,0,0
Vladimir Nabokov,1,0,0,1,0,0,1,0,0,0,...,0,0,5,3,0,1,0,1,1,1


In [2]:
# Import the necessary modiles for LDA with gensim

from gensim import matutils, models
import scipy.sparse
import gensim

D:\Annaconda\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
#### Transpose DataFrame. row to column, column to row
tdm.head()

,Ernest Hemingway,Joseph Conrad,Vladimir Nabokov
abilities,0,0,1
ability,1,0,0
able,4,0,0
abnormal,0,0,1
aboriginal,0,1,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of all terms and their respective location in the term-document matrix 
cv = pickle.load(open("cv_stop.pkl", 'rb'))
print(type(cv))
id2word = dict((v,k) for k,v in cv.vocabulary_.items())

<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [6]:
id2word

{384: 'carlos',
 224: 'baker',
 684: 'death',
 2661: 'summer',
 2385: 'school',
 630: 'critics',
 139: 'arisen',
 1295: 'holds',
 772: 'discuss',
 2214: 'really',
 2405: 'second',
 1648: 'major',
 3039: 'works',
 1952: 'path',
 161: 'artist',
 1193: 'gradual',
 2601: 'steady',
 724: 'descent',
 1293: 'hold',
 1879: 'opinions',
 256: 'belief',
 379: 'career',
 3051: 'writer',
 2905: 'ups',
 815: 'downs',
 2833: 'triumphs',
 1051: 'fiction',
 2759: 'thirty',
 3066: 'years',
 1572: 'life',
 2373: 'say',
 629: 'critical',
 320: 'brethren',
 1802: 'necessarily',
 3057: 'wrong',
 2079: 'preferring',
 1017: 'farewell',
 146: 'arms',
 1529: 'later',
 260: 'bell',
 2790: 'tolls',
 1866: 'old',
 1654: 'man',
 2398: 'sea',
 1647: 'maintain',
 586: 'continued',
 2: 'able',
 2072: 'practitioner',
 156: 'art',
 2153: 'publication',
 1846: 'obliged',
 779: 'disparage',
 3036: 'work',
 1885: 'order',
 42: 'admire',
 845: 'earlier',
 93: 'american',
 1836: 'novels',
 681: 'deal',
 3040: 'world',
 2970:

In [7]:
cv.vocabulary_

{'carlos': 384,
 'baker': 224,
 'death': 684,
 'summer': 2661,
 'school': 2385,
 'critics': 630,
 'arisen': 139,
 'holds': 1295,
 'discuss': 772,
 'really': 2214,
 'second': 2405,
 'major': 1648,
 'works': 3039,
 'path': 1952,
 'artist': 161,
 'gradual': 1193,
 'steady': 2601,
 'descent': 724,
 'hold': 1293,
 'opinions': 1879,
 'belief': 256,
 'career': 379,
 'writer': 3051,
 'ups': 2905,
 'downs': 815,
 'triumphs': 2833,
 'fiction': 1051,
 'thirty': 2759,
 'years': 3066,
 'life': 1572,
 'say': 2373,
 'critical': 629,
 'brethren': 320,
 'necessarily': 1802,
 'wrong': 3057,
 'preferring': 2079,
 'farewell': 1017,
 'arms': 146,
 'later': 1529,
 'bell': 260,
 'tolls': 2790,
 'old': 1866,
 'man': 1654,
 'sea': 2398,
 'maintain': 1647,
 'continued': 586,
 'able': 2,
 'practitioner': 2072,
 'art': 156,
 'publication': 2153,
 'obliged': 1846,
 'disparage': 779,
 'work': 3036,
 'order': 1885,
 'admire': 42,
 'earlier': 845,
 'american': 93,
 'novels': 1836,
 'deal': 681,
 'world': 3040,
 'war'

Now we have the corpus( term-document matrix) and id2word(dictionary of loaction:trem), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 5, see if the results make sense, and increase the number from there.

In [8]:
lda = models.LdaModel(corpus, id2word=id2word, num_topics=5, passes=10)
lda.print_topics()

[(0,
  '0.000*"time" + 0.000*"life" + 0.000*"years" + 0.000*"lolita" + 0.000*"english" + 0.000*"man" + 0.000*"like" + 0.000*"darkness" + 0.000*"love" + 0.000*"heart"'),
 (1,
  '0.000*"lolita" + 0.000*"english" + 0.000*"years" + 0.000*"time" + 0.000*"published" + 0.000*"world" + 0.000*"new" + 0.000*"version" + 0.000*"man" + 0.000*"love"'),
 (2,
  '0.015*"darkness" + 0.013*"heart" + 0.006*"dark" + 0.004*"narrator" + 0.004*"life" + 0.004*"kind" + 0.004*"adams" + 0.003*"story" + 0.003*"men" + 0.003*"title"'),
 (3,
  '0.008*"lolita" + 0.006*"russian" + 0.005*"english" + 0.004*"years" + 0.003*"time" + 0.003*"man" + 0.003*"write" + 0.003*"life" + 0.003*"world" + 0.003*"love"'),
 (4,
  '0.007*"war" + 0.006*"years" + 0.005*"time" + 0.005*"version" + 0.005*"farewell" + 0.004*"arms" + 0.004*"henry" + 0.004*"catherine" + 0.003*"love" + 0.003*"second"')]

## Topic Modeling Attempt #2 

In [9]:
from nltk import word_tokenize, pos_tag
def nouns(text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word,pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [10]:
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
Ernest Hemingway,by carlos baker since his death in the summer ...
Joseph Conrad,the ambiguous beginning of heart of darkness b...
Vladimir Nabokov,who and what is vladimir nabokov the author of...


In [11]:
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
Ernest Hemingway,carlos baker death summer school critics novel...
Joseph Conrad,beginning heart darkness adams adams title par...
Vladimir Nabokov,author lolita helen lawrenson aspects seller c...


In [12]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
add_stop_words = ['Ernest','Hemingway','Joseph','Conrad','Vladimir','Nabokov','nabokov','hemingway','book']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
#####print(data_cvn)   ##### (0, 274)	1
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,abilities,ability,abruzzi,absence,absorption,abstraction,accent,accents,acceptance,accident,...,yawl,year,years,yes,york,youth,zeitgeist,zoo,zoology,émigrés
Ernest Hemingway,0,1,2,0,0,0,0,1,1,1,...,0,1,17,1,3,0,1,0,0,0
Joseph Conrad,0,0,0,1,1,1,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
Vladimir Nabokov,1,0,0,0,0,0,1,0,0,0,...,0,5,11,0,5,0,0,1,1,1


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=5, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"lolita" + 0.007*"years" + 0.006*"time" + 0.006*"man" + 0.006*"vladimir" + 0.006*"life" + 0.006*"world" + 0.005*"wife" + 0.005*"professor" + 0.004*"university"'),
 (1,
  '0.011*"darkness" + 0.011*"heart" + 0.009*"years" + 0.009*"war" + 0.008*"time" + 0.007*"novel" + 0.007*"life" + 0.006*"story" + 0.006*"version" + 0.006*"farewell"'),
 (2,
  '0.001*"years" + 0.001*"novel" + 0.001*"time" + 0.001*"man" + 0.001*"life" + 0.001*"heart" + 0.001*"darkness" + 0.001*"world" + 0.001*"war" + 0.001*"lolita"'),
 (3,
  '0.001*"years" + 0.001*"time" + 0.001*"novel" + 0.001*"heart" + 0.001*"world" + 0.001*"darkness" + 0.001*"life" + 0.001*"version" + 0.001*"war" + 0.001*"farewell"'),
 (4,
  '0.001*"years" + 0.001*"time" + 0.001*"version" + 0.001*"war" + 0.001*"man" + 0.001*"world" + 0.001*"story" + 0.001*"life" + 0.001*"novel" + 0.001*"lolita"')]

## Attempt #3 (Nouns and Adjectives) 

In [16]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [17]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
Ernest Hemingway,carlos baker death summer school critics novel...
Joseph Conrad,ambiguous beginning heart darkness richard ada...
Vladimir Nabokov,vladimir author lolita helen lawrenson diverti...


In [18]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abilities,ability,able,abnormal,aboriginal,abrupt,abruzzi,absence,absorption,abstraction,...,yes,yoked,york,young,youth,youthful,zeitgeist,zoo,zoology,émigrés
Ernest Hemingway,0,1,4,0,0,0,2,0,0,0,...,1,0,3,8,0,0,1,0,0,0
Joseph Conrad,0,0,0,0,1,0,0,1,1,1,...,0,1,0,0,1,0,0,0,0,0
Vladimir Nabokov,1,0,0,1,0,1,0,0,0,0,...,0,0,5,3,0,1,0,1,1,1


In [19]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [20]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.001*"novel" + 0.001*"version" + 0.001*"war" + 0.001*"farewell" + 0.001*"darkness" + 0.001*"lolita" + 0.001*"hemingways" + 0.001*"american" + 0.001*"new" + 0.001*"heart"'),
 (1,
  '0.010*"darkness" + 0.009*"heart" + 0.008*"lolita" + 0.007*"russian" + 0.005*"nabokovs" + 0.004*"vladimir" + 0.004*"university" + 0.004*"professor" + 0.004*"french" + 0.003*"new"'),
 (2,
  '0.011*"novel" + 0.010*"war" + 0.009*"hemingways" + 0.008*"version" + 0.008*"farewell" + 0.005*"second" + 0.005*"rain" + 0.005*"catherine" + 0.005*"priest" + 0.005*"arms"'),
 (3,
  '0.001*"heart" + 0.001*"darkness" + 0.001*"novel" + 0.001*"lolita" + 0.001*"version" + 0.001*"war" + 0.001*"russian" + 0.001*"farewell" + 0.001*"new" + 0.001*"vladimir"'),
 (4,
  '0.001*"darkness" + 0.001*"novel" + 0.001*"lolita" + 0.001*"heart" + 0.001*"russian" + 0.001*"war" + 0.001*"nabokovs" + 0.001*"version" + 0.001*"french" + 0.001*"wife"')]

In [21]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(2, 'Ernest Hemingway'), (1, 'Joseph Conrad'), (1, 'Vladimir Nabokov')]

In [22]:
# Let's start with 2 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna,random_state=34,passes=25,per_word_topics=True)

In [32]:
ldamodel[corpusna]

In [55]:
sent_topics_df = pd.DataFrame()
for key,row_list in enumerate(ldamodel[corpusna]):
    row = row_list[0] if ldamodel.per_word_topics else row_list            
    print(row_list[2])
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:  # => dominant topic
            wp = ldamodel.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prop in wp])
            sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
        else:
            break
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']    


[(1, [(0, 0.99999964)]), (2, [(0, 4.0)]), (6, [(0, 1.9999998)]), (11, [(0, 0.99999964)]), (13, [(0, 0.99999964)]), (14, [(0, 0.99999964)]), (15, [(0, 0.9999997)]), (16, [(0, 2.9999948)]), (18, [(0, 5.0)]), (22, [(0, 1.9999998)]), (23, [(0, 0.9999996)]), (26, [(0, 0.99999964)]), (29, [(0, 0.99999964)]), (30, [(0, 0.99999696)]), (32, [(0, 0.9999997)]), (33, [(0, 0.9999997)]), (35, [(0, 0.999997)]), (36, [(0, 0.99999964)]), (37, [(0, 1.9999974)]), (38, [(0, 0.9999997)]), (42, [(0, 0.9999971)]), (44, [(0, 0.9999996)]), (46, [(0, 4.0)]), (47, [(0, 0.9999996)]), (48, [(0, 1.9999998)]), (50, [(0, 0.9999997)]), (53, [(0, 1.9999998)]), (54, [(0, 0.99999964)]), (60, [(0, 1.9999999)]), (61, [(0, 0.99999964)]), (63, [(0, 6.9999866)]), (67, [(0, 0.9999997)]), (68, [(0, 0.9999997)]), (76, [(0, 0.9999996)]), (79, [(0, 0.999997)]), (80, [(0, 0.99999964)]), (85, [(0, 0.9999996)]), (95, [(0, 0.99999964)]), (97, [(0, 8.999998)]), (98, [(0, 0.9999996)]), (100, [(0, 0.9999997)]), (104, [(0, 0.99999964)]), 

In [56]:
sent_topics_df

,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,0.0,0.9994,"novel, war, hemingways, version, farewell, arm..."
1,1.0,0.9987,"darkness, heart, lolita, russian, nabokovs, vl..."
2,1.0,0.9995,"darkness, heart, lolita, russian, nabokovs, vl..."
